In [ ]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import fashion_mnist,mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

In [ ]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NUM_CHANNELS = 1
MODEL_NAME = "FashionMNIST_rotation"
VOTING = 'HARD'
PATH = "/content/drive/My Drive/NC/"
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Preprocess

In [ ]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [ ]:
(x_train_val, y_train_val), (x_test, y_test) = fashion_mnist.load_data()

x_train_val = preprocess(x_train_val)
x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [ ]:
def FashionMNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [ ]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'soft'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

#Augmentation 


In [ ]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

# Train

In [ ]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"\n ===== Train model {i} =====")

        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # augmentation
        datagen = ImageDataGenerator(preprocessing_function=rotate_image)
        datagen = datagen.flow(x_train,y_train, batch_size= BATCH_SIZE)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)

        # Create the model
        model = FashionMNISTmodel(IMAGE_SIZE, NUM_CLASSES, 1)
        
        # Load the weighs if the model is already trained
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{i}.h5"

        if os.path.exists(weights_path):
            print(f"Skipping training of model {i}: weights exists")
            model.load_weights(weights_path)
        else:
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{i}.csv", separator=';')

            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val, y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            
            model.save_weights(weights_path)
        
        models.append(model)

        acc = predict(models, x_val, y_val, voting='hard')
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break

    # Results

    ## Accuracy vs nr of models
    ## Visualizing the accuracy vs the number of models in the ensamble

    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_{VOTING}.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc, voting='hard')
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_{VOTING}.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_{VOTING}.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)


 ===== Train model 1 =====
Skipping training of model 1: weights exists
Model: 1 added. Resulting score: 0.7575, Delta: 0.7575, Patience: 0

 ===== Train model 2 =====
Skipping training of model 2: weights exists
Model: 2 added. Resulting score: 0.7615, Delta: 0.0040000000000000036, Patience: 1

 ===== Train model 3 =====
Skipping training of model 3: weights exists
Model: 3 added. Resulting score: 0.773, Delta: 0.011500000000000066, Patience: 0

 ===== Train model 4 =====
Skipping training of model 4: weights exists
Model: 4 added. Resulting score: 0.775, Delta: 0.0020000000000000018, Patience: 1

 ===== Train model 5 =====
Skipping training of model 5: weights exists
Model: 5 added. Resulting score: 0.77425, Delta: -0.0007500000000000284, Patience: 2

 ===== Train model 6 =====
Epoch 1/9999
375/375 [==============================] - 8s 21ms/step - loss: 2.2037 - accuracy: 0.2144 - val_loss: 2.1809 - val_accuracy: 0.3212
Epoch 2/9999
375/375 [==============================] - 7s 19ms

,Accuracy,Nr of models
0,0.00000,0
1,0.75750,1
2,0.76150,2
3,0.77300,3
4,0.77500,4
5,0.77425,5
6,0.77525,6



 ===== Accuracy ======
Accuracy: 0.7684

 ===== Correlation =====


,0,1,2,3,4,5
0,NaN,0.738394,0.735429,0.758168,0.717971,0.739258
1,0.738394,NaN,0.777872,0.747763,0.734030,0.727013
2,0.735429,0.777872,NaN,0.780111,0.717294,0.719411
3,0.758168,0.747763,0.780111,NaN,0.710930,0.755744
4,0.717971,0.734030,0.717294,0.710930,NaN,0.709976
5,0.739258,0.727013,0.719411,0.755744,0.709976,NaN


Average correlation: 0.7379576472191507


# Results

In [ ]:
from scipy import stats

In [ ]:
baseline_hard =  [0.7774,
                  0.7774,
                  0.7764,
                  0.7717,
                  0.7669,
                  0.7729,
                  0.7743,
                  0.7662,
                  0.7743,
                  0.7735]

In [ ]:
baseline_soft=   [0.7795,
                  0.7814,
                  0.7824,
                  0.7747,
                  0.7753,
                  0.7758,
                  0.7768,
                  0.7722,
                  0.7776,
                  0.7742]

In [ ]:
np.mean(baseline_hard)

0.7731000000000001

In [ ]:
np.mean(baseline_soft)

0.77699

In [ ]:
augmented_hard = pd.read_csv(PATH+"FashionMNIST_rotation/results_HARD.csv")
augmented_hard = augmented_hard.values
augmented_hard

array([[ 1.        ,  0.7708    ,  0.74016156],
       [ 2.        ,  0.7725    ,  0.74890865],
       [ 3.        ,  0.7738    ,  0.73243286],
       [ 4.        ,  0.7679    ,  0.75930904],
       [ 5.        ,  0.7685    ,  0.74236975],
       [ 6.        ,  0.7652    ,  0.75432929],
       [ 7.        ,  0.7667    ,  0.75785838],
       [ 8.        ,  0.7671    ,  0.76382272],
       [ 9.        ,  0.7739    ,  0.7170636 ],
       [10.        ,  0.7684    ,  0.73795765]])

In [ ]:
augmented_soft = pd.read_csv(PATH+"FashionMNIST_rotation/results_SOFT.csv")
augmented_soft = augmented_soft.values
augmented_soft

array([[ 1.        ,  0.7703    ,  0.74380357],
       [ 2.        ,  0.7781    ,  0.74588633],
       [ 3.        ,  0.7798    ,  0.73243286],
       [ 4.        ,  0.7717    ,  0.75930904],
       [ 5.        ,  0.7719    ,  0.74236975],
       [ 6.        ,  0.7683    ,  0.75432929],
       [ 7.        ,  0.7713    ,  0.75689827],
       [ 8.        ,  0.774     ,  0.76382272],
       [ 9.        ,  0.7781    ,  0.7170636 ],
       [10.        ,  0.7672    ,  0.74179622]])

In [ ]:
augmented_hard = augmented_hard[:,1]
augmented_soft = augmented_soft[:,1]

In [ ]:
np.mean(augmented_hard)

0.7694799999999999

In [ ]:
np.mean(augmented_soft)

0.7730699999999999

In [ ]:
stats.ttest_rel(augmented_hard,baseline_hard)

Ttest_relResult(statistic=-3.3624385983932905, pvalue=0.008356725023494539)

In [ ]:
stats.ttest_rel(augmented_soft,baseline_soft)

Ttest_relResult(statistic=-3.575541823894351, pvalue=0.005972242419708263)

## Accuracy
The final accuracy of the ensamble on the test set